In [1]:
import pandas as pd
import pandas
import numpy as np
import glob
import csv
from sqlalchemy import create_engine


In [49]:
availability_df =pd.read_csv('calendar.csv')
listings_df=pd.read_csv('listings.csv')
review_df=pd.read_csv('reviews.csv')


In [50]:
listings_df['name']

0              Stylish Queen Anne Apartment
1        Bright & Airy Queen Anne Apartment
2       New Modern House-Amazing water view
3                        Queen Anne Chateau
4            Charming craftsman 3 bdm house
                       ...                 
3813     3BR Mountain View House in Seattle
3814      Portage Bay View!-One Bedroom Apt
3815      Private apartment view of Lake WA
3816      Amazing View with Modern Comfort!
3817              Large Lakefront Apartment
Name: name, Length: 3818, dtype: object

In [51]:
listing_df = listings_df[["id",'name',"street","neighbourhood_cleansed","neighbourhood_group_cleansed","city","state","zipcode","latitude","longitude","is_location_exact","property_type","room_type","accommodates","bathrooms","bedrooms","beds","bed_type","square_feet","price","weekly_price","monthly_price","security_deposit","cleaning_fee","guests_included","extra_people","minimum_nights","maximum_nights","has_availability","availability_30","availability_60","availability_90","availability_365","number_of_reviews","first_review","last_review","review_scores_rating","review_scores_accuracy","review_scores_cleanliness","review_scores_checkin","review_scores_communication","review_scores_location","review_scores_value","requires_license","instant_bookable","cancellation_policy","require_guest_profile_picture","require_guest_phone_verification","reviews_per_month","host_id"]].copy()
host_df = listings_df[["host_id","host_name","host_since","host_location","host_response_time","host_response_rate","host_acceptance_rate","host_is_superhost","host_neighbourhood","host_listings_count","host_has_profile_pic","host_identity_verified"]].copy()

In [52]:
print(listing_df.shape)
host_df.shape

(3818, 50)


(3818, 12)


# listings dataframe

In [53]:
# Remove duplicates
listing_df.drop_duplicates(keep="first",inplace=True)

# Convert columns to datetime format
listing_df["first_review"]=pd.to_datetime(listing_df["first_review"])
listing_df["last_review"]=pd.to_datetime(listing_df["last_review"])

# Replace columns designated as "t" or "f" as the "True" or "False" booleans
listing_df["is_location_exact"].replace(["t","f"],[True,False],inplace=True)
listing_df["has_availability"].replace(["t","f"],[True,False],inplace=True)
listing_df["requires_license"].replace(["t","f"],[True,False],inplace=True)
listing_df["instant_bookable"].replace(["t","f"],[True,False],inplace=True)
listing_df["require_guest_profile_picture"].replace(["t","f"],[True,False],inplace=True)
listing_df["require_guest_phone_verification"].replace(["t","f"],[True,False],inplace=True)

# Convert all currency columns to numeric values
currency_cols=["price","weekly_price","monthly_price","security_deposit","cleaning_fee","extra_people"]
listing_df[currency_cols]=listing_df[currency_cols].replace({'\$': '', ',': ''}, regex=True)
listing_df["price"] = pd.to_numeric(listing_df["price"])
listing_df["weekly_price"] = pd.to_numeric(listing_df["weekly_price"])
listing_df["monthly_price"] = pd.to_numeric(listing_df["monthly_price"])
listing_df["security_deposit"] = pd.to_numeric(listing_df["security_deposit"])
listing_df["cleaning_fee"] = pd.to_numeric(listing_df["cleaning_fee"])
listing_df["extra_people"] = pd.to_numeric(listing_df["extra_people"])

# hosts dataframe:

In [54]:
# Remove duplicates
host_df.drop_duplicates(keep="first",inplace=True)

# Convert columns to datetime format
host_df["host_since"]=pd.to_datetime(host_df["host_since"])

# Replace columns designated as "t" or "f" as the "True" or "False" booleans
host_df["host_is_superhost"].replace(["t","f"],[True,False],inplace=True)
host_df["host_has_profile_pic"].replace(["t","f"],[True,False],inplace=True)
host_df["host_identity_verified"].replace(["t","f"],[True,False],inplace=True)

# Convert all percentage columns to numeric values
host_df["host_response_rate"]=host_df["host_response_rate"].replace({'\$': '', ',': '', '%':''}, regex=True)
host_df["host_acceptance_rate"]=host_df["host_acceptance_rate"].replace({'\$': '', ',': '', '%':''}, regex=True)
host_df["host_response_rate"] = pd.to_numeric(host_df["host_response_rate"])
host_df["host_acceptance_rate"] = pd.to_numeric(host_df["host_acceptance_rate"])

# property availability dataframe

In [55]:
availability_df

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN
...,...,...,...,...
1393565,10208623,2016-12-29,f,NaN
1393566,10208623,2016-12-30,f,NaN
1393567,10208623,2016-12-31,f,NaN
1393568,10208623,2017-01-01,f,NaN


In [56]:
# Convert columns to datetime format
availability_df["date"]=pd.to_datetime(availability_df["date"])

# Replace columns designated as "t" or "f" as the "True" or "False" booleans
availability_df["available"].replace(["t","f"], [True,False], inplace=True)

# Convert all currency columns to numeric values
availability_df["price"] = availability_df["price"].replace({'\$': '', ',': ''}, regex=True)
availability_df["price"] = pd.to_numeric(availability_df["price"])

# reviews dataframe

In [57]:
review_df

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...
...,...,...,...,...,...,...
84844,3624990,50436321,2015-10-12,37419458,Ryan,The description and pictures of the apartment ...
84845,3624990,51024875,2015-10-17,6933252,Linda,We had an excellent stay. It was clean and com...
84846,3624990,51511988,2015-10-20,19543701,Jaime,"Gran ubicación, cerca de todo lo atractivo del..."
84847,3624990,52814482,2015-11-02,24445024,Jørgen,"Very good apartement, clean and well sized. Si..."


In [58]:
# Convert columns to datetime format
review_df["date"] = pd.to_datetime(review_df["date"])

# Establish the connection with SQL Server

In [59]:
pip install pyodbc

In [33]:
# conn_str=("Driver={ODBC Driver 17 for SQL Server};"
#  "Server=DESKTOP-B1JT9V8;"
#  "Database= Airbnb;"
# "Trusted_Connection=yes;")
# pyodbc.connect(conn_str)

In [ ]:
import pandas as pd
import pyodbc

# Define your SQL Server connection parameters
server = 'your_server_name'
database = 'your_database_name'
username = 'your_username'
password = 'your_password'

# Define the path to your CSV file
csv_file = 'your_file.csv'

# Establish a connection to SQL Server
conn_str = f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

try:
    # Read CSV file into pandas DataFrame
    host_df = pd.read_csv(csv_file)

    # Preprocess data: replace incorrect decimal separators
    host_df['host_response_rate'] = host_df['host_response_rate'].str.replace(',', '.')

    # Attempt bulk insert operation into SQL Server with error logging
    for index, row in host_df.iterrows():
        try:
            # Construct and execute the insert query
            cursor.execute("INSERT INTO YourTableName (host_response_rate) VALUES (?)", row['host_response_rate'])
            conn.commit()
        except pyodbc.Error as e:
            # Log the error message
            print(f"Error inserting row {index+1}: {e}")

except Exception as ex:
    print(f"An error occurred: {ex}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()


In [13]:
# import pandas as pd
# import pyodbc

# # Establish connection string
# server = 'DESKTOP-B1JT9V8'

# database = 'Airbnb'
# username = 'nada'
# password = '123'

# driver = '{ODBC Driver 17 for SQL Server}'
# #connection_string = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
# connection_string = f'mssql://{username}:{password}@{server}/{database}?driver={driver}'



# # Name of the table to which you want to write
# table_name = 'airbnb_hosts'

# try:
#     # Create SQLAlchemy engine
#     engine = create_engine(connection_string)
#     # Establish connection
#     #conn = pyodbc.connect(connection_string)
# # Write DataFrame to SQL Server using the engine
#     host_df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

#     # Write DataFrame to SQL Server
#     #host_df.to_sql(name=table_name, con=conn, if_exists='replace', index=False)

#     print("DataFrame written to SQL Server successfully!")

# except Exception as e:
#     print("An error occurred:", e)

# finally:
#     # Close connection
#     conn.close()


In [66]:
review_df.rename(columns={'date': 'review_date'}, inplace=True)
availability_df.rename(columns={'date': 'availability_date'}, inplace=True)
listing_df.rename(columns={'name': 'listing_name'}, inplace=True)


In [15]:
host_df.to_csv('host.csv')
review_df.to_csv('review.csv')
availability_df.to_csv('availability.csv')
listing_df.to_csv('listing.csv')


In [36]:

# host_df['host_acceptance_rate'].fillna(0,inplace=True)

# host_df['host_acceptance_rate'].isnull().sum()


0

In [63]:
host_df=host_df.dropna()

In [68]:
host_df['host_listings_count'].isnull().sum()

0

In [81]:
#host_df.astype({'host_acceptance_rate': 'int64','host_response_rate':'int64','host_listings_count':'int64','host_identity_verified': 'bool','host_has_profile_pic': 'bool','host_is_superhost': 'bool'}).dtypes
host_df['host_acceptance_rate'] = host_df['host_acceptance_rate'].astype('int32')
host_df['host_response_rate'] = host_df['host_response_rate'].astype('int32')
host_df['host_listings_count'] = host_df['host_listings_count'].astype('int32')
host_df['host_identity_verified'] = host_df['host_identity_verified'].astype('bool')
host_df['host_has_profile_pic'] = host_df['host_has_profile_pic'].astype('bool')
host_df['host_is_superhost'] = host_df['host_is_superhost'].astype('bool')


In [ ]:
review_df.to_csv('review.csv',header=True,sep=',', index=False, encoding='utf-8')
availability_df.to_csv('availability.csv',header=True,sep=',', index=False, encoding='utf-8')
listing_df.to_csv('listing.csv',header=True,sep=',', index=False, encoding='utf-8')

In [ ]:

# BULK INSERT dbo.Table_1
# FROM '‪G:\SQL\sql_project\cleaned_data'
# WITH
# FORMAT='xls', FIRSTROW=2
# GO

In [83]:
host_df.to_csv('host.csv',header=True,sep=',', index=False, encoding='utf-8')